In [ ]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, TFBertModel
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import torch

In [6]:
MAX_SEQUENCE_LENGTH = 300
MAX_NUM_WORDS = 50000

In [7]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)

In [16]:
from keras.models import load_model
model = load_model('C:/Users/ADMIN PC/Desktop/Comment/CNN/CNN.h5')

In [17]:
def predict_proba(arr):
  sequences_new = tokenizer.texts_to_sequences(arr)
  data = pad_sequences(sequences_new, maxlen=MAX_SEQUENCE_LENGTH)
  yprob = model.predict(data, verbose=None)
  # yclasses=yprob.argmax(axis=-1)

  returnable=[]
  for i in yprob:
    temp=i[0]
    returnable.append(np.array([temp,1- temp]))
  return np.array(returnable)

In [18]:
def predict_male_or_female(txt):
    arr = np.array([txt])
    sequences_new = tokenizer.texts_to_sequences(arr)
    data = pad_sequences(sequences_new, maxlen=MAX_SEQUENCE_LENGTH)
    yprob = model.predict(data)[0]
    gender = "Female" if yprob[1] >= 0.5 else "Male"

    return gender

In [ ]:
txt = "I've had the 50watter since Oct of last year and I'm still impressed and blown away every time I play it. It's unreal. Killer choice"
predict_male_or_female(txt)

In [ ]:
txt = "You l've got a good man there hun. Take care of each other and it'll last a long time.\nFor sore throats my dad used to take 2 tablespoons of apple cider vinegar and the same amount of honey, mix it in at least 8oz of hot water. Drink it while ot's still hot, but not burning. Worked every time for me...still does."
predict_male_or_female(txt)

In [21]:
from lime.lime_text import LimeTextExplainer, IndexedString
from operator import itemgetter
from tqdm import tqdm
import csv

In [22]:
class_names = ['Male', 'Female']
explainer = LimeTextExplainer(class_names=class_names)

In [ ]:
txt = "I've had the 50watter since Oct of last year and I'm still impressed and blown away every time I play it. It's unreal. Killer choice"
explainer.explain_instance(txt, predict_proba).show_in_notebook(text=True)

In [ ]:
txt = "You l've got a good man there hun. Take care of each other and it'll last a long time.\nFor sore throats my dad used to take 2 tablespoons of apple cider vinegar and the same amount of honey, mix it in at least 8oz of hot water. Drink it while ot's still hot, but not burning. Worked every time for me...still does."
explainer.explain_instance(txt, predict_proba).show_in_notebook(text=True)

In [26]:
def sort_tuples_array_by_second_item(tuples):
    return sorted(tuples, key=itemgetter(1))

In [27]:
# Function to get max explained words
def get_max_explained_words(txt, explainer_num_samples=100):
    prediction = predict_male_or_female(txt)
    prediction_label = 0 if prediction == "Male" else 1

    exp = explainer.explain_instance(txt, predict_proba, num_samples=explainer_num_samples)
    exp_list = []
    for x in zip(exp.local_exp[1], exp.as_list()):
        exp_list.append((x[1][0], x[1][1], x[0][0]))

    # Features with negative score are for Male class
    male_list = list(filter(lambda x: x[1] < 0, exp_list))
    male_list = sort_tuples_array_by_second_item(male_list)

    # Features with positive score are for Female class
    female_list = list(filter(lambda x: x[1] > 0, exp_list))
    female_list = sort_tuples_array_by_second_item(female_list)

    # If comment predicted Male
    if prediction_label == 0:
        if len(male_list) > 1:
            male_mc = male_list[0]
            if (male_mc[0], 0) in words:
                words[(male_mc[0], 0)]['lime_score'].append(male_mc[1])
                words[(male_mc[0], 0)]['position'] = male_mc[2]
            else:
                words[(male_mc[0], 0)] = {}
                words[(male_mc[0], 0)]['lime_score'] = [male_mc[1]]
                words[(male_mc[0], 0)]['position'] = male_mc[2]
                wordsForCSV.append([male_mc[0], 0, male_mc[1]])

            male_mc = male_list[1]
            if (male_mc[0], 0) in words:
                words[(male_mc[0], 0)]['lime_score'].append(male_mc[1])
                words[(male_mc[0], 0)]['position'] = male_mc[2]
            else:
                words[(male_mc[0], 0)] = {}
                words[(male_mc[0], 0)]['lime_score'] = [male_mc[1]]
                words[(male_mc[0], 0)]['position'] = male_mc[2]
                wordsForCSV.append([male_mc[0], 0, male_mc[1]])
    else:
        if len(female_list) > 1:
            female_mc = female_list[(len(female_list) - 1)]
            if (female_mc[0], 1) in words:
                words[(female_mc[0], 1)]['lime_score'].append(female_mc[1])
                words[(female_mc[0], 1)]['position'] = female_mc[2]
            else:
                words[(female_mc[0], 1)] = {}
                words[(female_mc[0], 1)]['lime_score'] = [female_mc[1]]
                words[(female_mc[0], 1)]['position'] = female_mc[2]
                wordsForCSV.append([female_mc[0], 1, female_mc[1]])

            female_mc = female_list[(len(female_list) - 2)]
            if (female_mc[0], 1) in words:
                words[(female_mc[0], 1)]['lime_score'].append(female_mc[1])
                words[(female_mc[0], 1)]['position'] = female_mc[2]
            else:
                words[(female_mc[0], 1)] = {}
                words[(female_mc[0], 1)]['lime_score'] = [female_mc[1]]
                words[(female_mc[0], 1)]['position'] = female_mc[2]
                wordsForCSV.append([female_mc[0], 1, female_mc[1]])

    return words, wordsForCSV

In [ ]:
def load_original_data(data_file):
    df = pd.read_csv(data_file)
    df.fillna("", inplace=True)  
    comments = df['comment'].tolist()
    genders = df['user_gender'].map(lambda x: 0 if x == "Male" else 1).tolist()
    return comments, genders

In [ ]:
original_comments, original_genders = load_original_data('C:/Users/ADMIN PC/Desktop/Comment/Comment_dataset.csv')

In [30]:
n = int(len(original_comments) / 7)
x = [original_comments[i:i + n] for i in range(0, len(original_comments), n)]
y = [original_genders[i:i + n] for i in range(0, len(original_genders), n)]

In [ ]:
header = ["word", "label", "limescore"]
file_path = 'C:/Users/ADMIN PC/Desktop/Comment/CNN/file.csv'
words = {}
wordsForCSV = []

In [ ]:
for comment in tqdm(x[0], total=len(x[0])):
    words, wordsForCSV = get_max_explained_words(comment)

with open(file_path, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(wordsForCSV)

In [ ]:
for comment in tqdm(x[1], total=len(x[1])):
    words, wordsForCSV = get_max_explained_words(comment)

with open(file_path, 'a', encoding='UTF8', newline='') as f:  
    writer = csv.writer(f)
    writer.writerows(wordsForCSV)  

In [ ]:
for comment in tqdm(x[2], total=len(x[2])):
    words, wordsForCSV = get_max_explained_words(comment)

with open(file_path, 'a', encoding='UTF8', newline='') as f:  
    writer = csv.writer(f)
    writer.writerows(wordsForCSV)  

In [ ]:
for comment in tqdm(x[3], total=len(x[3])):
    words, wordsForCSV = get_max_explained_words(comment)

with open(file_path, 'a', encoding='UTF8', newline='') as f:  
    writer = csv.writer(f)
    writer.writerows(wordsForCSV)  

In [ ]:
for comment in tqdm(x[4], total=len(x[4])):
    words, wordsForCSV = get_max_explained_words(comment)

with open(file_path, 'a', encoding='UTF8', newline='') as f:  
    writer = csv.writer(f)
    writer.writerows(wordsForCSV)

In [ ]:
for comment in tqdm(x[5], total=len(x[5])):
    words, wordsForCSV = get_max_explained_words(comment)

with open(file_path, 'a', encoding='UTF8', newline='') as f:  
    writer = csv.writer(f)
    writer.writerows(wordsForCSV)

In [ ]:
for comment in tqdm(x[6], total=len(x[6])):
    words, wordsForCSV = get_max_explained_words(comment)

with open(file_path, 'a', encoding='UTF8', newline='') as f:  
    writer = csv.writer(f)
    writer.writerows(wordsForCSV)  